In [18]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [19]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [20]:
sample = pd.read_csv('sample.csv')

In [21]:
import re
def text(text):
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    text = text.lower()
       
    return text

In [22]:
train["clean_text"]=train.tweet.apply(lambda x: text(x))
test["clean_text"]=test.tweet.apply(lambda x: text(x))

In [23]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"
training_size = 20000
padding_type = 'post'

In [24]:
train_sentences = train.clean_text
test_sentences = test.clean_text
train_labels = train.label


In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
token = Tokenizer(num_words=vocab_size,oov_token = oov_tok)
token.fit_on_texts(train_sentences)
word_index = token.word_index

In [26]:
train_seq = token.texts_to_sequences(train_sentences)
train_pad = pad_sequences(train_seq, maxlen = max_length,truncating = trunc_type,padding = padding_type)
test_seq = token.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_seq, maxlen = max_length,truncating = trunc_type,padding = padding_type)

In [27]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [28]:
import numpy as np
train_labels = np.array(train_labels)
#test_labels = np.array(test_labels)
num_epochs = 15
history = model.fit(train_pad, train_labels, epochs=num_epochs,verbose = 2)

Train on 7920 samples
Epoch 1/15
7920/7920 - 2s - loss: 0.5686 - accuracy: 0.7442
Epoch 2/15
7920/7920 - 1s - loss: 0.4305 - accuracy: 0.7628
Epoch 3/15
7920/7920 - 1s - loss: 0.2897 - accuracy: 0.8809
Epoch 4/15
7920/7920 - 1s - loss: 0.2441 - accuracy: 0.8986
Epoch 5/15
7920/7920 - 1s - loss: 0.2239 - accuracy: 0.9045
Epoch 6/15
7920/7920 - 1s - loss: 0.2078 - accuracy: 0.9149
Epoch 7/15
7920/7920 - 1s - loss: 0.1928 - accuracy: 0.9236
Epoch 8/15
7920/7920 - 1s - loss: 0.1803 - accuracy: 0.9287
Epoch 9/15
7920/7920 - 1s - loss: 0.1667 - accuracy: 0.9345
Epoch 10/15
7920/7920 - 1s - loss: 0.1542 - accuracy: 0.9420
Epoch 11/15
7920/7920 - 1s - loss: 0.1438 - accuracy: 0.9470
Epoch 12/15
7920/7920 - 1s - loss: 0.1316 - accuracy: 0.9520
Epoch 13/15
7920/7920 - 1s - loss: 0.1221 - accuracy: 0.9574
Epoch 14/15
7920/7920 - 1s - loss: 0.1139 - accuracy: 0.9602
Epoch 15/15
7920/7920 - 1s - loss: 0.1064 - accuracy: 0.9639


In [29]:
predictions = model.predict(test_pad)

In [30]:
pred = np.round(predictions).astype('int').flatten()

In [31]:
# sub = pd.DataFrame({'id':test.id,'label':pred})
# sub.to_csv('submission_3.csv',index = False )